# Building an image retrieval system with deep features


# Fire up GraphLab Create
dataset train https://d396qusza40orc.cloudfront.net/phoenixassets/image_train_data.csv  
dataset test https://d396qusza40orc.cloudfront.net/phoenixassets/image_test_data.csv

In [1]:
import numpy as np
import pandas as pd

# Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [2]:
image_train =pd.read_csv('image_train_data.csv')
image_test = pd.read_csv('image_test_data.csv')

# Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [3]:
# deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
# image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [4]:
image_train.head()

,id,image,label,deep_features,image_array
0,24,Height: 32 Width: 32,bird,[0.242872 1.09545 0 0.39363 0 0 11.8949 0 0 0 ...,[73 77 58 71 68 50 77 69 44 120 116 83 125 120...
1,33,Height: 32 Width: 32,cat,[0.525088 0 0 0 0 0 9.94829 0 0 0 0 0 1.01264 ...,[7 5 8 7 5 8 5 4 6 7 4 7 11 5 9 11 5 9 17 11 1...
2,36,Height: 32 Width: 32,cat,[0.566016 0 0 0 0 0 9.9972 0 0 0 1.38345 0 0.7...,[169 122 65 131 108 75 193 196 192 218 221 222...
3,70,Height: 32 Width: 32,dog,[1.1298 0 0 0.778194 0 0.758051 9.83053 0 0 0....,[154 179 152 159 183 157 165 189 162 174 199 1...
4,90,Height: 32 Width: 32,bird,[1.71787 0 0 0 0 0 9.33936 0 0 0 0 0 0.412137 ...,[216 195 180 201 178 160 210 184 164 212 188 1...


# Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [5]:
from sklearn.neighbors import KNeighborsClassifier

In [6]:
image_train['deep_features']=image_train['deep_features'].apply(lambda x:[float(i) for i in x[1:-1].split(' ')])
image_test['deep_features']=image_test['deep_features'].apply(lambda x:[float(i) for i in x[1:-1].split(' ')])

In [7]:
train_deep_features = np.array([i for i in image_train['deep_features'].values])
train_y = image_train.id.values

In [8]:
knn_model = KNeighborsClassifier()
knn_model.fit(train_deep_features,train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [9]:
#knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
  #                                           label='id')

# Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [10]:
#graphlab.canvas.set_target('ipynb')
cat = train_deep_features[18:19]
#cat['image'].show()

In [11]:
dist,ind = knn_model.kneighbors(cat)
label = image_train['label'][ind[0]]

In [12]:
pd.DataFrame({'ind':ind[0],'dist':dist[0],'label':label})

,dist,ind,label
18,0.000000,18,cat
288,36.940312,288,cat
1565,38.463489,1565,cat
1468,39.755967,1468,cat
1633,39.786597,1633,cat


We are going to create a simple function to view the nearest neighbors to save typing:

In [13]:
def get_images_from_ids(query_result):
    dist,ind = query_result
    label = image_train['label'][ind[0]]
    return pd.DataFrame({'ind':ind[0],'dist':dist[0],'label':label})

In [14]:
cat_neighbors = get_images_from_ids(knn_model.kneighbors(cat))

In [15]:
cat_neighbors.head()

,dist,ind,label
18,0.000000,18,cat
288,36.940312,288,cat
1565,38.463489,1565,cat
1468,39.755967,1468,cat
1633,39.786597,1633,cat


Very cool results showing similar cats.

## Finding similar images to a car

In [16]:
car = train_deep_features[8:9]
#car['image'].show()

In [17]:
get_images_from_ids(knn_model.kneighbors(car)).head()

,dist,ind,label
8,0.000000,8,automobile
372,32.310817,372,automobile
1757,33.925375,1757,automobile
1343,35.023130,1343,automobile
1009,35.376827,1009,automobile


# Just for fun, let's create a lambda to find and show nearest neighbor images

In [18]:
show_neighbors = lambda i: get_images_from_ids(knn_model.kneighbors(train_deep_features[i:i+1])).head()

In [19]:
show_neighbors(8)

,dist,ind,label
8,0.000000,8,automobile
372,32.310817,372,automobile
1757,33.925375,1757,automobile
1343,35.023130,1343,automobile
1009,35.376827,1009,automobile


In [20]:
show_neighbors(26)

,dist,ind,label
26,0.000000,26,automobile
457,34.715476,457,automobile
377,35.793145,377,automobile
1576,36.665344,1576,automobile
1280,36.878414,1280,automobile


# Question 1

In [21]:
image_train['label'].describe()

count     2005
unique       4
top        dog
freq       509
Name: label, dtype: object

In [22]:
image_train['label'].value_counts()

dog           509
cat           509
automobile    509
bird          478
Name: label, dtype: int64

# Question 2

In [23]:
dog = train_deep_features[(image_train.label=='dog').values]
cat = train_deep_features[(image_train.label=='cat').values]
automobile = train_deep_features[(image_train.label=='automobile').values]
bird = train_deep_features[(image_train.label=='bird').values]

In [24]:
dog_y = image_train.id[image_train.label=='dog'].values
cat_y = image_train.id[image_train.label=='cat'].values
automobile_y = image_train.id[image_train.label=='automobile'].values
bird_y = image_train.id[image_train.label=='bird'].values

In [25]:
dog_model = knn_model.fit(dog,dog_y)
cat_model = knn_model.fit(cat,cat_y)
automobile_model = knn_model.fit(automobile,automobile_y)
bird_model = knn_model.fit(bird,bird_y)

In [26]:
test_deep_features = np.array([i for i in image_test['deep_features'].values])
test_y = image_test.id.values
image_test[0:1].head()

,id,image,label,deep_features,image_array
0,0,Height: 32 Width: 32,cat,"[1.13469, 0.0, 0.0, 0.0, 0.0366498, 0.0, 9.353...",[158 112 49 159 111 47 165 116 51 166 118 53 1...


In [27]:
get_images_from_ids(cat_model.kneighbors(test_deep_features[0:1]))

,dist,ind,label
474,38.074267,474,automobile
447,39.101527,447,automobile
169,39.699105,169,cat
44,39.703980,44,dog
1,39.890882,1,cat


In [28]:
get_images_from_ids(dog_model.kneighbors(test_deep_features[0:1]))

,dist,ind,label
474,38.074267,474,automobile
447,39.101527,447,automobile
169,39.699105,169,cat
44,39.703980,44,dog
1,39.890882,1,cat


# Question 3

In [29]:
get_images_from_ids(cat_model.kneighbors(test_deep_features[0:1])).dist.mean()

39.29395209827638

In [30]:
get_images_from_ids(dog_model.kneighbors(test_deep_features[0:1])).dist.mean()

39.29395209827638

# Question 4

In [31]:
image_test_dog = test_deep_features[(image_test.label=='dog').values]
image_test_cat = test_deep_features[(image_test.label=='cat').values]
image_test_automobile = test_deep_features[(image_test.label=='automobile').values]
image_test_bird = test_deep_features[(image_test.label=='bird').values]